# 分布密度估算的采样法学习笔记
全文来源：https://blog.51cto.com/13811808/2130740

## 如何采样

有了概率分布P(X)，我们要生成符合这个分布的样本。这个过程也叫采样(sampling)。

这其实并不是一个简单的任务。我们知道，计算机可以生成符合某些指定分布的伪随机数（伪随机数即样本），但并不支持任意的分布。对于所支持的非均匀分布的伪随机数x的产生，我们需要先产生均匀分布的伪随机数y，然后通过某种变换，把y转换成指定的非均匀分布的x。例如下图，y轴上是一些均匀分布的点，通过某个函数Fx映射到x轴，就得到非均匀分布的点了。如果函数Fx足够复杂，我们可以把任意一个分布转换成另一个分布。如何在两个分布之间找到这样的函数，这是VAE的关键点之一。

均匀分布的伪随机数的生成常用两个算法，它们都是用数列迭代的方法产生伪随机数。一个叫线性同余序列(Linear Congruential Generator, LCG)，公式是
$$n_k=a*n_{k-1}+b mod M$$
![Image](Images/2019_3_6_1.jpg)
产生[0, M]之间的一个随机数。a或b，以及M要足够大才能有好的结果。早期的Matlab使用的是这个算法：a=75，b=0， M=231 – 1。

n0是初始数，称为"种子"(seed)。若要重现实验结果，在产生随机数时要提供使用的seed。

另一个叫Lagged Fibonacci Generator，公式是
$$n_k=a*n_{k-}+b*n_{k-} + C mod M$$
新版的Matlab (>R5.0) 使用的是这个算法 ：a=1，b=1， =17，=5。

In [3]:
seed = 0.1
a = 7**5
b = 0
M = 2**31-1
for i in range(10):
    if i == 0:
        n = seed
    else:
        n = a*n + b%M
        print(n)

1680.7
28247524.900000002
474756150994.30005
7979226629761201.0
1.341068619663965e+20
2.2539340290692262e+24
3.7881869226566484e+28
6.366805760909029e+32
1.0700690442359804e+37


In [5]:
seed = 0.1
a = 1
b = 1
C = 17
M = 5
for i in range(10):
    if i == 0:
        n = seed
    else:
        n = a*n + b*n + C %M
        print(n)

2.2
6.4
14.8
31.6
65.2
132.4
266.8
535.6
1073.2


均匀分布到非均匀分布的转换方法也有两种，一种可以称为CDF Transformation，另一种可以称为PDF Transformation。
前者需要知道随机变量的CDF(Cumulative Distribution Function)，即离散变量
$$F_X(x) = P\{X\leq x\}$$
连续变量
$$F_X(x) = \int_{-\infty}^{x}f(x)dx$$
先在均匀分布上采集一个随机变量u，则
$$Z = F_X^{-1}(u)$$
就是符合Fx分布的随机变量，即Z具有与X相同的分布。$F_X^{-1}$是$F_X$的逆函数。
证明过程很简单。Z的CDF是$F_Z(z)$，
$$F_z(z) = P(Z \leq z) = P\{ F_X^{-1}(u) \leq z\}$$
证毕。

显然，只有当$F_x^{-1}$有解析解时，CDF Transformation才是可行的。
比如，
$$F_X(x) = 1 + \frac{1}{1+e^{-x}}$$
则
$$Z = F_X^{-1}(u) = ln(\frac{1}{F_X(u)}-1)$$
如果我们知道的是x的概率密度函数f(x)，为了使用上述方法，我们要先计算它的CDF：
$$F_X(x) = \int_{-\infty}^{x}f(x)dx$$
这一步也无法保证有解析解。下面是一个有解析解的例子：
$$f(x) = \gamma e^{-\gamma x}$$
则
$$F_X(x) =1 - e^{-\gamma x}$$

In [6]:
# CDF 模拟

第二种方法是PDF Transformation，直接利用概率密度函数（PDF）推导出函数h，使得X = h(U)把均匀分布的样本U转换成需要的X的分布。这个方法避免求积分，但需要求h-1的雅可比矩阵，看上去也不是一个容易使用的方法。我们常用的标准高斯分布(x) ~ N(0, 1)的伪随机数的产生方法是Box-Muller公式，其推导就使用了PDF Transformation，但用了一个巧妙的方法，即极坐标方法，避开了实际求雅可比矩阵，这里就不展开了。下图直观地解释了这种方法的原理：每个圈（从浅绿色到深绿色）里的数据点的数量基本一样（在极坐标里是均匀分布的），但在x-y坐标系里就是高斯分布的了（点的密集度从中央到边缘越来越低，其实在x-轴和y-轴上的分布都是高斯分布
![image](Images/2019_3_6_2.jpg)
Polar method is sometimes more convenient to think of generating a pair of random variates (x, y) from the target distribution p(z). We can map this pair to a representation in polar form (rcosθ, rsinθ), which exposes other mechanisms for sampling,   e.g., the famous Box-Muller transform is  derived in this way.
来源：http://blog.shakirm.com/2015/10/machine-learning-trick-of-the-day-4-reparameterisation-tricks/
对于标准正态（高斯）分布$N(0, 1)$，另一种采样方法是利用中心极限定理，从二值分布（Bernoulli分布）大量地采集样本$x_1,x_2,\dots,x_n$，然后计算出一个高斯随机数：
$$N(0,1)\approx 2 \sqrt{n}(\frac{x_1+x_2+\cdots+x_n}{n}-0.5)$$
问题是，这个方法需要很长的计算时间，因为每个高斯随机数的产生都要先产生n个Bernoulli分布的随机数。因此一般我们还是用Box-Muller公式，直接把均匀分布转换成标准高斯分布。

至此，我们知道如何对一些常见的有公式表达的概率分布进行采样了。我们记住这些方法的思路，就是先得到均匀分布的样本，再通过一个函数把这些样本转换成所需的分布的样本。进一步延伸，我们应该可以把从已知分布（不一定是均匀分布）采样的样本，通过一个函数，转换成我们需要的任意分布。这里，我们的任务就是找到这个函数。

In [7]:
#PDF 模拟实验

## 蒙特卡洛方法
那么，如果概率分布f(x)是任意的，同时CDF Transformation和PDF Transformation都很难或无法找到解析解呢？这时我们一般使用一个直截了当的采样方法，名为MCMC（Markov Chain Monte Carlo）的采样方法，中文翻译成马尔科夫链-蒙特卡罗方法，就是在马尔科夫链上跑蒙特卡罗方法，所以它有两个部分，即蒙特卡罗方法和马尔科夫链。
先简要介绍一下蒙特卡罗方法（这时你的头脑中是否闪现过一个念头：也许还有拉斯×××方法？）。这个方法的最初目的是求定积分：
$$ I = \int_{\Omega}f(x)dx$$
其中，是定积分的x取值域。这个积分问题的难处在于函数f(x)很复杂，或者x的维度很高，使得这个定积分没有或很难找到解析解，常规的数值方法也不适用。这时我们可以用采样的方法来求I的近似值。先看个x为一维的简单例子（下图来自Wikipedia） ：
![image](Images/2019_3_6_3.jpg)
我们来求的近似值。方法是在上图的正方形里撒均匀分布的N个点（红+蓝），计算出落入1/4圆里面有C个点（红）。正方形的面积是1，则圆的面积是
$$\frac{4C}{N}=\pi r^2 = \pi$$
![image](Images/2019_3_6_4.jpg)
对单一的一维变量x的函数f(x)做定积分，就是求f(x)的曲线下的面积。无论f(x)的曲线多么复杂，我们都可以把需要积分的这段放在一个正方形里，再用上述方法计算曲线下的面积。假如正方形的面积是S，N个均匀采样点{(xi, yi), i=1..N}中有C个点满足
$$y_i \leq f(x_i)$$
则f(x)下的面积约是
$$\frac{4C}{N}\times S$$
当N足够大时，这个面积就近似于f(x)的定积分。这就是基本的蒙特卡罗方法。

这个方法需要在坐标系的横轴和纵轴方向以正方形为界，采集均匀分布的多对点{(xi, yi), i=1..N}。如果x是多维变量，那就需同时在各个维上采集均匀样本点。均匀分布的采样我们在上面已经介绍了。

这种计算定积分的近似值的方法称为Rejection Sampling。意思是，N个采样点中，只有落在f(x)曲线下的区域里的C个样本点才被接受（上图中红色的点），在这个区域之外的点（蓝色的点）被拒绝。

那么，定积分的这种近似解法和任意分布f(x)的采样有什么关系 ？因为CDF的计算公式是

我们可以用"拒绝"的方法把

内的均匀采样得到的样本点{(xi, yi), i=1..N}检查一遍，只留下满足

的那些xi点作为符合f(x)分布的样本点，其它的点都被拒绝。这样，在f(x)越大的地方，留下的样本点越多，因为这些地方的f(x)下的面积也大。最后留下的{xi}就是合法的样本点。这也算是一种从均匀分布到任意分布的转换方法。
这个方法的缺点是，如果那个正方形与f(x)之间的间隙比较大，比如当f(x)的曲线有 "陡峭的山峰"形状，则被拒绝的采样点也很多，造成较大的浪费（获得这些采样点所花的计算资源和时间）。
提高效率的一个方法是Importance Sampling，意思是，f(x)取值大的地方（即"尖锐的山峰"处）所取样本的点数应该多于f(x)取值小的地方，即越重要的地方，样本点应该越多，越密集。这也是概率密度的本意。下面是一个例子，来自http://bjlkeng.github.io/posts/markov-chain-monte-carlo-mcmc-and-the-metropolis-hastings-algorithm/。

图中蓝线的Target（即f(x)）是双伽马分布(Double Gamma Distribution)。可以看到，它有尖锐的"山峰"。如果我们围绕它画一个矩形（假如我们知道f(x)的峰值），并在这个矩形里进行均匀采样，那么被拒绝的点会非常多（落入蓝线条以上的区域的点）。但如果我们能用一个Envelope函数q(x)覆盖f(x)，然后在横坐标方向以q(x)为分布采样xi，在纵坐标方向继续以均匀分布采样yi，那么被拒绝的点就会少很多，因为大多数xi被q(x)集中在中间"山峰"的位置。当然，q(x)的选择是个关键，它必须满足两个特点，一是它足够简单，容易采样，二是能大致符合f(x)的形状，使q(x)和 f(x)之间的空隙尽可能地小。图中，黑色虚线q(x)是高斯分布N (0, 2)。在f(x)的取值大的区域，从q(x)生成的样本xi也聚集得多。前面的内容已经讲过如何从高斯分布采样（Box-Muller公式）。这样，我们可以用一个简短的程序就能实现双伽马分布的Importance Sampling。

注意，来自q(x)的样本xi被接受的概率等于f(xi)/q(xi)，即f(xi)和q(xi)的高度之比。这是因为在纵向的采样是在0 ~ q(xi)之间的均匀采样，样本点落入f(xi)以下部分的概率是f(xi)/q(xi)。请仔细观察图中的绿虚线和红虚线之间的关系。Importance Sampling也是一种Rejection Sampling，只不过在横轴方向的采样不再是均匀分布的，而是按照q(x)的分布采样。

让我们再用数学语言来描述一下Importance Sampling。